# Model Denemeleri

### Önce HuggingFace'ten, hali hazırda tamamen türkçe veriler üzerinde eğitilmiş BerTurk modeli denenecek

In [1]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForPreTraining.from_pretrained("dbmdz/bert-base-turkish-cased")

In [2]:
sd_hf = model.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [8]:
torch.equal(sd_hf["bert.embeddings.word_embeddings.weight"], sd_hf["cls.predictions.decoder.weight"]) # tying kontrolü

True

In [7]:
sd_hf["bert.embeddings.word_embeddings.weight"] is sd_hf["cls.predictions.decoder.weight"]  # diffrent memory, but same values (because of tying) (when tying we cannot just directly share tensors (same memory), because of compile/torchscript does not like it)

False

In [5]:
torch.equal(sd_hf["cls.predictions.decoder.bias"], sd_hf["cls.predictions.bias"]) # tying kontrolü

True

In [9]:
sd_hf["cls.predictions.decoder.bias"] is sd_hf["cls.predictions.bias"]  # diffrent memory, but same values (because of tying) (when tying we cannot just directly share tensors (same memory), because of compile/torchscript does not like it)

False

In [4]:
inputs = tokenizer("Selamlar efendim", return_tensors="pt")
outputs = model(**inputs)

In [5]:
outputs.prediction_logits.shape, outputs.seq_relationship_logits.shape

(torch.Size([1, 4, 32000]), torch.Size([1, 2]))

In [7]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]